In [4]:
# 1. Iris Flower Classification

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
iris = load_iris()

In [4]:
X = iris.data
y = iris.target

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.2, random_state=42)

In [6]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [7]:
X_train_tensor = torch.FloatTensor(X_train)
y_train_tensor = torch.LongTensor(y_train)
X_test_tensor = torch.FloatTensor(X_test)
y_test_tensor = torch.LongTensor(y_test)

In [8]:
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

In [10]:
class IrisClassifier(nn.Module):
  def __init__(self):
    super(IrisClassifier, self).__init__()
    self.fc1 = nn.Linear(4, 16)
    self.fc2 = nn.Linear(16, 3)

  def forward(self, x):
    x = torch.relu(self.fc1(x))
    x = self.fc2(x)
    return x

model = IrisClassifier()

In [11]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [12]:
epochs = 50

for epoch in range(epochs):
  model.train()
  running_loss = 0.0

  for inputs, labels in train_loader:
    optimizer.zero_grad()

    outputs = model(inputs)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

    running_loss += loss.item()

    print(f"Epoch {epoch+1}, loss: {running_loss/len(train_loader)}")

Epoch 1, loss: 0.1648169755935669
Epoch 1, loss: 0.31817902624607086
Epoch 1, loss: 0.4583896994590759
Epoch 1, loss: 0.6056092232465744
Epoch 1, loss: 0.7491379827260971
Epoch 1, loss: 0.8848067671060562
Epoch 1, loss: 1.0207010358572006
Epoch 1, loss: 1.1510581523180008
Epoch 2, loss: 0.12807951867580414
Epoch 2, loss: 0.25663962960243225
Epoch 2, loss: 0.37775523215532303
Epoch 2, loss: 0.49501923471689224
Epoch 2, loss: 0.610022060573101
Epoch 2, loss: 0.7241707444190979
Epoch 2, loss: 0.8305681943893433
Epoch 2, loss: 0.9357649981975555
Epoch 3, loss: 0.10318322479724884
Epoch 3, loss: 0.20683591067790985
Epoch 3, loss: 0.2980787381529808
Epoch 3, loss: 0.39201752841472626
Epoch 3, loss: 0.4949055388569832
Epoch 3, loss: 0.579279363155365
Epoch 3, loss: 0.6608472168445587
Epoch 3, loss: 0.7242841050028801
Epoch 4, loss: 0.07330384105443954
Epoch 4, loss: 0.15410290658473969
Epoch 4, loss: 0.23519383370876312
Epoch 4, loss: 0.3050651103258133
Epoch 4, loss: 0.3574351780116558
Epoch

In [13]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
  for inputs, labels in test_loader:
    outputs = model(inputs)
    _, predicted = torch.max(outputs, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()

print(f"Accuracy: {100 * correct / total}%")

Accuracy: 100.0%


In [15]:
import numpy as np

random_samples = torch.FloatTensor(np.random.rand(5, 4))

random_samples_standardized = torch.FloatTensor(scaler.transform(random_samples))

model.eval()
with torch.no_grad():
  predictions = model(random_samples_standardized)
  _, predicted_classes = torch.max(predictions, 1)

print("Random Samples (Standardized):")
print(random_samples_standardized)
print("Predicted Classes:")
print(predicted_classes)

Random Samples (Standardized):
tensor([[-6.7248, -6.5508, -1.5820, -0.9180],
        [-6.8447, -4.6840, -1.6838, -0.8634],
        [-6.9513, -5.8968, -1.9581, -1.4037],
        [-6.8328, -5.2141, -1.7056, -1.3768],
        [-6.9149, -5.4712, -1.6365, -1.5281]])
Predicted Classes:
tensor([0, 0, 0, 0, 0])
